In [1]:
%matplotlib inline
import pandas as pd
import zipfile
import shutil
import urllib2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates


from urllib2 import urlopen     
import time

from bs4 import BeautifulSoup
from pylab import rcParams
import platform
rcParams['figure.figsize'] = 15, 10
import re
import os

import glob
import urllib

import gzip

import ftplib
import calendar
import datetime
from datetime import date

import pymodis

In [2]:
import arcpy
arcpy.CheckOutExtension("spatial")
#from arcpy import env 
from arcpy.sa import *

In [3]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Pandas Version " + str(pd.__version__))
print("Numpy Version " + str(np.__version__))

Operating System Windows 7
Python Version 2.7.10 (default, May 23 2015, 09:44:00) [MSC v.1500 64 bit (AMD64)]
Pandas Version 0.19.2
Numpy Version 1.12.0


# MODIS16

Potential and Actual Evapotranspiration

https://github.com/lucadelu/pyModis/blob/master/docs/source/examples/pyModis.ipynb<br>
http://www.ntsg.umt.edu/project/mod16

In [ ]:
dest = 'H:/GIS/MODIS'

## Download HDF Files

The following script automatically retrieves monthly MODIS16 hdf file from the ntsg website.

In [ ]:
#Download HDF Files
#'h09v05','h09v04','h08v05'
tiles = ['h09v05','h09v05','h09v04','h08v05']
save_path = 'H:/GIS/MODIS2/'


def get_modis(tiles, save_path, months='', years=''):

    if months == '':
        months = [1,12]
    if years == '':
        years = [2000,2015]

    mons = [str(i).zfill(2) for i in range(months[0],months[1]+1)]
    yrs = [str(i) for i in range(years[0],years[1]+1)]

    for tile in tiles:
        for yr in yrs:
            for m in mons:
                base_url = "http://files.ntsg.umt.edu/data/NTSG_Products/MOD16/MOD16A2_MONTHLY.MERRA_GMAO_1kmALB/"

                dir_path = "Y{:}/M{:}/".format(yr,m)
                url = base_url+dir_path
                soup=BeautifulSoup(urlopen(url),"lxml")
                hdf_name = soup.find_all('', {'href':re.compile('MOD16A2.A{:}M{:}.{:}.105'.format(yr,m,tile), re.IGNORECASE)})
                files = urllib.urlretrieve(url+hdf_name[0].text, save_path + hdf_name[0].text)
                print(save_path+hdf_name[0].text)
                time.sleep(0.5)
                
get_modis(tiles, save_path)

In [ ]:
save_path = 'H:/GIS/MODIS2/'
files = glob.glob(os.path.join(save_path, '*.105*.hdf'))

def get_file_list(save_path):
    return glob.glob(os.path.join(save_path, '*.105*.hdf'))

get_file_list(save_path)

## Reproject MODIS Data

These scripts reproject the hdf files from the <a href="https://modis-land.gsfc.nasa.gov/MODLAND_grid.html">wacky sinusoidal MODIS projection</a> to <a href="http://spatialreference.org/ref/epsg/4269/">NAD83 Zone 12</a> for analysis.  In this section, the MODIS rasters are also clipped to Utah Watersheds (see image below) and the fill values are made to null values. Fill values are described in the <a href="http://files.ntsg.umt.edu/data/NTSG_Products/MOD16/MOD16_global_evapotranspiration_description.pdf">MODIS16 documentation</a>:
<br>
<ul>
<li>Fill value, out of the earth 32767</li>
<li>Water body 32766</li>
<li>Barren or sparsely vegetated 32765</li> 
<li>Permanent snow and ice 32764</li>
<li>Permanent wetland 32763</li>
<li>Urban or Built-up 32762</li>
<li>Unclassified 32761</li>
<img src="https://cfpub.epa.gov/surf/images/states/ut.gif">

The EPSG code for NAD83 Zone 12 is 26912.

In [ ]:
def reprojectMODIS(files,save_path,data_type,proj=26912):
    """Iterates through MODIS files in a folder reprojecting them. 
    
    Takes the crazy MODIS sinusoidal projection to a user defined projection.
    
    Args:
        files: list of file paths of MODIS hdf files; created using files = glob.glob(os.path.join(save_path, '*.105*.hdf'))
        save_path: folder to store the reprojected files
        data_type: type of MODIS16 data being reprojected; options are 'ET','PET','LE', and 'PLE'
        
    Returns:
        Reprojected MODIS files
        
    """
    # dictionary to designate a directory
    datadir = {'ET':'/ET/','PET':'/PET/','LE':'/LE/','PLE':'/PLE/'}
    # dictionary to select layer from hdf file that contains the datatype
    matrdir = {'ET':[1,0,0,0], 'LE':[0,1,0,0], 'PET':[0,0,1,0], 'PLE':[0,0,0,1]}
    
    # check for file folder and make it if it doesn't exist
    if not os.path.exists(save_path + datadir[data_type]):
        os.makedirs(save_path + datadir[data_type])
        print('created {:}'.format(save_path + datadir[data_type]))
    
    
    for f in files:
        year = f.split('\\')[1].split('.')[1][1:5] # parse year from hdf filename
        month = f.split('\\')[1].split('.')[1][-2:] # parse month from hdf filename
        v = f.split('\\')[1].split('.')[2][-2:] # parse v (cell coordinate) from hdf filename
        h = f.split('\\')[1].split('.')[2][1:3] # parse h (cell coordinate) from hdf filename
        pref = os.path.join(save_path+datadir[data_type]+'A'+ year+'M'+ month +'h'+ h + 'v' + v)
        convertsingle = pymodis.convertmodis_gdal.convertModisGDAL(hdfname=f, prefix=pref, 
                                                                   subset = matrdir[data_type], 
                                                                   res=1000, epsg=proj)
        #[ET,LE,PET,PLE]
        try:
            convertsingle.run()
        except:
            print('A'+ year+'M'+ month +'h'+ h + 'v' + v + ' failed!')
            pass

def clipandfix(path, outpath, data_type, area = ''):
    """Clips raster to Utah's Watersheds and makes exception values null.
    
    Args:
        path: folder of the reprojected MODIS files
        outpath: ESRI gdb to store the clipped files
        data_type: type of MODIS16 data being reprojected; options are 'ET','PET','LE', and 'PLE'
    
    """
    # Check out the ArcGIS Spatial Analyst extension license
    arcpy.CheckOutExtension("Spatial")

    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True

    if area == '':
        area = 'H:/GIS/NHD_UT_Proj.gdb/UT_HUC_area'
    
    arcpy.env.mask = area

    for rast in arcpy.ListRasters():
        calc = SetNull(arcpy.Raster(rast) > 32760, arcpy.Raster(rast)) 
        calc.save(outpath+data_type+rast[1:5]+rast[6:8]+'h'+rast[10:11]+'v'+rast[13:14])
        print(outpath+data_type+rast[1:5]+rast[6:8]+'h'+rast[10:11]+'v'+rast[13:14])

### Reproject ET

In [ ]:
reprojectMODIS(files,save_path,'ET')

In [ ]:
path = "H:/GIS/MODIS2/ET/"
outpath="H:/GIS/MODIS2/MODIS.gdb/"
clipandfix(path,outpath,'ET')

### Reproject PET

In [ ]:
save_path = "H:/GIS/MODIS2/PET/"
reprojectMODIS(files,save_path,'PET')

In [ ]:
path = "H:/GIS/MODIS2/PET/"
outpath="H:/GIS/MODIS2/MODIS.gdb/"
clipandfix(path,outpath,'PET')

## Merge Data

MODIS data is downloaded as separate cells based on the Sinusoidal grid.  Three MODIS cells cover Utah ('h09v05','h09v04','h08v05'). The following scripts mosaic (merge) the three individual rasters for each month into one seamless monthly raster for the entire state. 
<img src="https://modis-land.gsfc.nasa.gov/images/MODIS_sinusoidal_grid1.gif" alt="MODIS grid">

In [ ]:
def mergeRasts(path, data_type = 'ET', monthRange = [1,12], yearRange = [2000,2014]):
    """Mosaics (merges) different MODIS cells into one layer.
    
    
    """
    arcpy.env.workspace = path
    outCS = arcpy.SpatialReference('NAD 1983 UTM Zone 12N')
    for y in range(yearRange[0],yearRange[-1]+1): #set years converted here
        for m in range(monthRange[0],monthRange[-1]+1): #set months converted here   
            nm = data_type + str(y) + str(m).zfill(2)
            rlist=[]
            for rast in arcpy.ListRasters(nm+'*'): 
                rlist.append(rast)
            try:
                arcpy.MosaicToNewRaster_management(rlist,path,nm+'c',outCS,\
                                                   "16_BIT_UNSIGNED","1000","1","LAST","LAST")
            
                print(path+nm+'c')
            except:
                print(nm+' failed!')
                pass

### Merge ET

In [ ]:
path="H:/GIS/MODIS2/MODIS.gdb/"
mergeRasts(path,data_type='ET', monthRange = [1,12], yearRange = [2011,2014])

### Merge PET

In [ ]:
path="H:/GIS/MODIS2/MODIS.gdb/"
mergeRasts(path,data_type='PET')

## Scale MODIS Data

The documentation for the dataset says that the raw files have to be multiplied by 0.1 to get mm/month.  To convert to meters per month, we have to multiply the raw files by 0.0001 (0.1 x 0.001) or divide by 10,000.  These scripts do that division. 

In [ ]:
def scale_MODIS(path, out_path, scaleby = 10000.0, data_type = 'ET', monthRange = [1,12], yearRange = [2000,2014]):
    for y in range(yearRange[0],yearRange[-1]+1): #set years converted here
        for m in range(monthRange[0],monthRange[-1]+1): #set months converted here
            nm = data_type + str(y) + str(m).zfill(2)
            calc = Divide(nm + 'c', scalingFactor)
            calc.save(out_path+nm)
        

In [ ]:
path="H:/GIS/MODIS2/MODIS.gdb/"
arcpy.env.workspace = path

scalingFactor = 10000.0 #convert to m/month
out_path="H:/GIS/MODIS2/MODIS.gdb/"

scale_MODIS(path, out_path, scaleby = 10000.0, data_type = 'ET')
scale_MODIS(path, out_path, scaleby = 10000.0, data_type = 'PET')

## Clean up GDB

This deletes intermediate files left over from previous processing steps. For whatever reason, it take a while.

In [ ]:
path="H:/GIS/MODIS2/MODIS.gdb/"
arcpy.env.workspace = path

for rast in arcpy.ListRasters('*c'):
    print(rast)
    arcpy.Delete_management(rast, 'RasterDataset')
    
#for rast in arcpy.ListRasters('*h*v*'):
#    print(rast)
#    arcpy.Delete_management(rast, 'RasterDataset')

## Fill Holes in Rasters

This processing step fills in null values created when the fill values were removed [above](#Reproject-MODIS-Data).

From: http://gis.stackexchange.com/questions/136075/fill-in-nodata-gaps-in-raster-using-arcgis-for-desktop

In [ ]:
path= "H:/GIS/MODIS2/MODIS.gdb/"
outpath = "H:/GIS/MODIS2/MODIS16.gdb/"

units = 'CELL'
radius = 15

arcpy.env.workspace = path

def fill_holes(path, outpath, wildcard, units='CELL', radius=15):
    for rast in arcpy.ListRasters(wildcard):
        rfilled = arcpy.sa.Con(arcpy.sa.IsNull(rast),
                              arcpy.sa.FocalStatistics(rast,
                                                       arcpy.sa.NbrCircle(radius, units),'MEAN'), rast)
        dsc = arcpy.Describe(rast)
        nm = dsc.baseName
        rfilled.save(outpath+nm)
        print(nm)
    

# SNODAS

Data were downloaded using polaris: http://nsidc.org/data/polaris/

Alternatively, you can download from the ftp, which is slower. <br>
ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02158/ <br>
https://support.nsidc.org/entries/64231694-FTP-Client-Data-Access 

### Download SNODAS TARs


In [ ]:
L = u'H:\GIS\SNODAS'
def get_snodas(out_file,months='',years=''):
    if months == '':
        months = [1,12]
    if years == '':
        years = [2000,2015]

    monnames = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    mons = [str(i).zfill(2) + "_" + monnames[i-1] for i in range(months[0],months[1]+1)]

    yrs = [str(i) for i in range(years[0],years[1]+1)]

    for yr in yrs:
        for m in mons:
            ftp_addr = "sidads.colorado.edu"
            ftp = ftplib.FTP(ftp_addr)
            ftp.login()

            dir_path = "pub/DATASETS/NOAA/G02158/masked/" + yr + "/" + m + "/"
            ftp.cwd(dir_path)
            files = ftp.nlst()

            for f in files:
                if len(f) > 4:
                    save_file = open(L + "/" + f,'wb')
                    ftp.retrbinary("RETR "+f, save_file.write)
                    save_file.close()
                    print(f)
            ftp.close()

### Extract SNODAS Data Files

<br>http://nsidc.org/data/docs/noaa/g02158_snodas_snow_cover_model/<br>

In [ ]:
#https://github.com/PSU-CSAR/SNODAS-SWE/blob/master/snodas-swe-prep.py


Unzip tar files and put in unzip file

In [ ]:
snodasTars = glob.glob(u'H:/GIS/SNODAS/*.tar')
for tared in snodasTars:
    untar(tared,u'H:/GIS/SNODAS/SNODASUNZIPPED/')

Ungz gz files and delete gz files

In [ ]:
unsnodasTars = glob.glob(u'H:/GIS/SNODAS/SNODASUNZIPPED/*.gz')
for tared in unsnodasTars:
    ungz(tared, deletesource=True)

Replace header file to make compatible with ArcGIS

In [ ]:
for hdrfile in glob.glob("H:/GIS/SNODAS/SNODASUNZIPPED/*.Hdr"):
    replace_hdr_file(hdrfile)

## Polaris Download

Downloaded data from http://nsidc.org/data/polaris/, which allows for clipping to an area, as well as output as GeoTiff Format.  The download takes about 3 days and the unzipping takes about 2 hours.

<b>Extent:</b><br>
<ul type="disc">
<li>Top `44.0254000046`</li>
<li>Left `-116.075366662`</li>
<li>Right `-108.375366657`</li>
<li>Bottom `36.0087333333`</li>

<b>Selected Variables:</b> <p>Snow Melt Runoff at the Base of the Snow Pack, Sublimation of Blowing Snow, Snow Pack Average Temperature, Sublimation from the Snow Pack, Snow Water Equivalent, Liquid Precipitation, Snow Depth, Solid Precipitation</p><br>
<b>Download Format:</b> GeoTIFF<br>
<b>Output Grid:</b> 30" (Cylindrical Equidistant)

After unzipping files, rename them to make them easier to handle.

From http://nsidc.org/data/docs/noaa/g02158_snodas_snow_cover_model/, the file abbreviations are as follows:
<ul type="disc">
<li>`RAIN` = `Wet Precip`</li>
<li>`SWEQ` = `Snow Water Equivalent`</li> 
<li>`SNOD` = `Snow Depth`</li> 
<li>`SPAT` = `Snow Pack Average Temp`</li>
<li>`BSSB` = `Blowing Snow Sublimation`</li>
<li>`SNML` = `Snowmelt`</li>
<li>`SPSB` = `Snow Pack Sublimation`</li>
</ul>

In [ ]:
def rename_polaris_snodas(path):
    prodcode = {'us_ssmv11038wS__A':'SPAT', 'us_ssmv11044bS__T':'SNML', 'us_ssmv11050lL00T':'SPSB', 
                'us_ssmv11034tS__T':'SWEQ', 'us_ssmv01025SlL00':'RAIN', 'us_ssmv01025SlL01':'SNOW',
                'us_ssmv11036tS__T':'SNOD', 'us_ssmv11039lL00T':'BSSB'}

    #path = "H:/GIS/SNODAS/SNWDS"
    for filename in os.listdir(path):
        if filename.startswith("us_ssmv"):
            code = prodcode[filename[0:17]]
            yrsrt = filename.find('TNATS') + 5
            yr = filename[yrsrt:yrsrt+4]
            mo = filename[yrsrt+4:yrsrt+6]
            dy = filename[yrsrt+6:yrsrt+8]
            try:
                os.rename(os.path.join(path, filename), os.path.join(path,code+yr+mo+dy+filename[-4:]))
            except:
                pass
            
path = "C:/Users/PAULINKENBRANDT/Downloads/NSIDC_Data"           
rename_polaris_snodas(path)

## Data Merge (daily to monthly)

This merge uses geoTiff files from Polaris

https://pro.arcgis.com/en/pro-app/help/analysis/spatial-analyst/mapalgebra/building-complex-statements.htm
    

In [ ]:
def snowsummary(code, scalingFactor, statistics="SUM", outcellsize = '1000',monthRange = [1,12], yearRange = [2003,2016], 
                path="H:/GIS/SNODAS/SNWDS/", outpath="H:/GIS/SNODAS.gdb/"):
    '''
    summarizes daily SNODAS data to monthly values
    
    INPUT
    -----
    code = text; prefix of dataset to use; choices are 'RAIN','SWEQ','SNOD','SPAT','BSSB','SNML', or 'SPSB'
    scalingFactor = float; table 1 at http://nsidc.org/data/docs/noaa/g02158_snodas_snow_cover_model/
    statistics = text; from arcpy sa CellStatistics; choices are MEAN, MAJORITY, MAXIMUM, MEDIAN, MINIMUM, MINORITY, 
                    RANGE, STD, SUM, or VARIETY
    monthRange = len 2 list; begin and end month of data you wish to analyze
    yearRange = len 2 list; bengin and end year of data you wish to analyze
    path = directory where raw geoTiffs are located
    outpath = directory where final data will be stored
    
    OUTPUT
    ------
    projected and scaled monthly rasters

    '''
    g = {}
    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True
    
    statstype = {'MEAN':'AVG', 'MAJORITY':'MAJ', 'MAXIMUM':'MAX', 'MEDIAN':'MED', 'MINIMUM':'MIN', 'MINORITY':'MNR', 
                 'RANGE':'RNG', 'STD':'STD', 'SUM':'SUM', 'VARIETY':'VAR'}

    for y in range(yearRange[0],yearRange[1]+1): #set years converted here
        for m in range(monthRange[0],monthRange[1]+1): #set months converted here
            g[code+str(y)+str(m).zfill(2)] = [] #this defines the dictionary key based on data type month and year
            for name in sorted(glob.glob(path+code+'*.tif')): #pick all tiff files from raw data folder of a data type
                rast = os.path.basename(name) 
                if rast[0:4] == code and int(rast[4:8]) == y and int(rast[8:10]) == m:
                    g[code+str(y)+str(m).zfill(2)].append(rast) #create a list of rasters for each month
                else:
                    pass
            if len(g[code+str(y)+str(m).zfill(2)])>0:
                print(g[code+str(y)+str(m).zfill(2)])
                # arcpy sa functions that summarize the daily data to monthly data
                calc = CellStatistics(g[code+str(y)+str(m).zfill(2)], statistics_type = statistics, ignore_nodata="DATA")
                calc = Divide(calc, scalingFactor) #scale factor, converts to kg/m2 10 then to m 0.001
                calc = Con(calc < 0.0,0.0,calc) #remove negative and null values
                calc = Con(IsNull(calc),0, calc) #remove null
                outCS = arcpy.SpatialReference('NAD 1983 UTM Zone 12N') #change coordinate units to m for spatial analysis
                outnm = outpath+rast[0:4]+str(y).zfill(2)+str(m).zfill(2)+statstype[statistics]
                arcpy.ProjectRaster_management(calc, outnm, outCS, 'BILINEAR', outcellsize,
                                               'WGS_1984_(ITRF00)_To_NAD_1983', '#', '#')

### Precipitation

Includes snow and rain, which are provided as separate data sets in the SNOTEL data.

#### Wet Precipitation

This is a measure of the cumulative incoming rain over a month.<br> 
Units are meters of water per month.

In [ ]:
snowsummary('RAIN', 10000.0, "SUM", monthRange=[1, 12], yearRange=[2004, 2014])

#### SNOW

This represents the "dry" precipitation that falls to the ground as snow.<br>
We summed daily data to create cumulative monthly snowfall.<br>
Units are in meters of water per month.

In [ ]:
snowsummary('SNOW', 10000.0, "SUM", monthRange=[9, 9], yearRange=[2016, 2016])

#### Total Precipitation

In [ ]:
monthRange = [9,9] 
yearRange = [2016,2016]

g = {}
path="H:/GIS/SNODAS/SNODASproj.gdb/"
arcpy.env.workspace = path
arcpy.env.overwriteOutput = True

for y in range(yearRange[0],yearRange[1]+1): #set years converted here
    for m in range(monthRange[0],monthRange[1]+1): #set months converted here
        my = str(y)+str(m).zfill(2)
        newdn = 'TPPT' + my
        try:
            calc = Plus('SNOW'+ my +'SUM', 'RAIN'+ my +'SUM')
            calc.save(newdn+'SUM')
            print(newdn)
        except(RuntimeError):
            pass

### SWE

SWE = Snow-water equivalent;  This is a measure of the estimated total water content of the snow pack. <br>
The median is used to summarize these data.<br>
Units are in meters of water.

In [ ]:
snowsummary('SWEQ', 1000.0, 'MEDIAN', monthRange=[9, 9], yearRange=[2016, 2016])

### Snowmelt

SNML = Cumulative monthly snowmelt calculated by summing daily snowmelt data.<br>
Units are in meters of water per month.
Snow Melt Runoff at the Base of the Snow Pack.
Units are meters of water per month.

In [ ]:
snowsummary('SNML', 100000.0, 'SUM', monthRange=[1, 12], yearRange=[2004, 2014])

### Snow Sublimation

BSSB = Cumulative monthly blowing snow sublimation.<br>
SPSB = Cumulative monthly snow pack sublimation.<br>
TSSB = Total Cumulative monthly snow sublimation.<br>
Units are in meters of water per month.

In [ ]:
snowsummary('BSSB', 100000.0, 'SUM', monthRange=[9, 9], yearRange=[2016, 2016])

In [ ]:
snowsummary('SPSB', 100000.0, 'SUM', monthRange=[9, 9], yearRange=[2016, 2016])

In [ ]:
g = {}
path="H:/GIS/SNODAS/SNODASproj.gdb/"
arcpy.env.workspace = path
arcpy.env.overwriteOutput = True

monthRange = [9,9] 
yearRange = [2016,2016]


for y in range(yearRange[0],yearRange[1]+1): #set years converted here
    for m in range(monthRange[0],monthRange[1]+1): #set months converted here
        my = str(y)+str(m).zfill(2)
        newdn = 'TSSB' + my
        try:
            calc = Plus('BSSB'+ my +'SUM', 'SPSB'+ my +'SUM')
            calc.save(newdn+'SUM')
            print(newdn)
        except(RuntimeError):
            pass

## Monthly Averages

In [ ]:
def totalavg(code, statistics="MEAN", monthRange = [1,12], yearRange = [2003,2016], 
                path="H:/GIS/SNODAS/SNODASproj.gdb/", outpath="H:/GIS/SNODAS/SNODASproj.gdb/"):
    '''
    Summarizes daily raster data into monthly data.
    
    INPUT
    -----
        code = string with four letters represting data type to summarize (example 'BSSB')
        statistics = how data will be summarized; defaults to monthly averages; options are
            ['MEAN','MAJORITY','MAXIMUM','MEDIAN','MINIMUM','MINORITY','RANGE','STD','SUM','VARIETY']
            Most common are 'MEAN','MEDIAN', and 'SUM'
            These are inputs that will be used in the ArcPy CellStatistics function. 
            See http://pro.arcgis.com/en/pro-app/tool-reference/spatial-analyst/cell-statistics.htm for documentation
        monthRange = beginning and end months of summary statistics
        yearRange = beginning and end years of summary statistics
        path = location of geodatabase of data to summarize
        outpath = location of geodatabase where output data should be stored
    OUTPUT
    ------
        summary raster(s) stored in outpath
    
    '''
    g = {}
    statstype = {'MEAN':'AVG', 'MAJORITY':'MAJ', 'MAXIMUM':'MAX', 'MEDIAN':'MED', 'MINIMUM':'MIN', 'MINORITY':'MNR', 
                 'RANGE':'RNG', 'STD':'STD', 'SUM':'SUM', 'VARIETY':'VAR'}
    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True
    
    #iterate over month range set here; default is 1 to 12 (Jan to Dec)
    for m in range(monthRange[0],monthRange[1]+1): 
        
        #this defines the dictionary key based on data type, month, and year
        g[code+'0000'+str(m).zfill(2)] = [] 
        
        #pick all tiff files from raw data folder of a data type
        for rast in arcpy.ListRasters():
            yrrng = range(yearRange[0],yearRange[1]+1) #set years converted here
            
            # create a list of rasters with the right code and month and year
            if rast[0:4] == code and int(rast[4:8]) in yrrng and int(rast[8:10]) == m:
                g[code+'0000'+str(m).zfill(2)].append(rast) #create a list of rasters for each month
            else:
                pass
        if len(g[code+'0000'+str(m).zfill(2)])>0:
            
            # arcpy sa functions that summarize the daily data to monthly data
            calc = CellStatistics(g[code+'0000'+str(m).zfill(2)], statistics_type = statistics, ignore_nodata="DATA")
            calc.save(code+'0000'+str(m).zfill(2)+statstype[statistics])
            print(code+'0000'+str(m).zfill(2)+statstype[statistics])

In [ ]:
totalavg('TSSB')

In [ ]:
totalavg('SNML', monthRange=[2, 12], yearRange=[2003, 2016])

# PRISM

PRISM Data
http://www.prism.oregonstate.edu/

<a href=http://www.prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf>PRISM Web Services</a>

In [ ]:
monthRange = [1,12]

fileplace = 'H:/GIS/PRISM/PRISM'

for m in range(monthRange[0],monthRange[1]+1): 
    testfile = urllib.URLopener()
    testfile.retrieve("http://services.nacse.org/prism/data/public/normals/800m/ppt/"+ str(m).zfill(2), 
                      fileplace+str(m).zfill(2)+'.zip')


In [ ]:
testfile = urllib.URLopener()
testfile.retrieve("http://services.nacse.org/prism/data/public/normals/800m/ppt/"+ '14', 
                    fileplace+'14'+'.zip')

In [ ]:
fileplace = 'H:/GIS/PRISM/PRISM/'

In [ ]:
def unzipper(filepath):

    import zipfile

    f = zipfile.ZipFile(filepath,'r')
    f.extractall(filepath[:-6])
 

In [ ]:
for files in glob.glob(fileplace+'*.zip'):
    unzipper(files)
    print(files)

In [ ]:
monthRange = [1,12]

fileplace = 'H:/GIS/PRISM/PRISM'

for m in range(monthRange[0],monthRange[1]+1): 
    testfile = urllib.URLopener()
    testfile.retrieve("http://services.nacse.org/prism/data/public/normals/800m/ppt/"+ str(m).zfill(2), 
                      fileplace+str(m).zfill(2)+'.zip')
testfile = urllib.URLopener()
testfile.retrieve("http://services.nacse.org/prism/data/public/normals/800m/ppt/"+ 14, 
                    fileplace+str(m).zfill(2)+'.zip')

# Calculations

## Available Water

Calculates monthly available water:
    $$Available\;water = Rain + Snowmelt$$
<br>

In [ ]:
path="H:/GIS/SNODAS.gdb/"
path2 = "U:/GWP/Groundwater/Projects/BCM/Data/AvailableWater2.gdb/"

def calc_avail_water(path, path2, months='',years='')

    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True

    if months == '':
        months = [1,12] 
    if years == '':
        years = [2004,2014]

    for y in range(years[0], years[1]+1): #set years converted here
        for m in range(months[0], months[1]+1): #set months converted here
            my = str(y)+str(m).zfill(2)
            newdn = 'AVWT' + my
            rain = Raster('RAIN'+ my +'SUM')
            melt = Raster('SNML'+ my +'SUM')
            calc = rain + melt
            calc.save(path2+newdn)
            print(newdn)
        
calc_avail_water(path, path2)

## Available Soil Water

<p>The calculation of excess water provides the water that is available for watershed hydrology. Available water occupies the soil profile, where water will become actual evapotranspiration,and may result in runoff or recharge, depending on the permeability of the underlying bedrock. Total soil‐water storage is calculated as porosity multiplied by soil depth. Field capacity (soil water volume at ‐0.3 megapascals [MPa]) is the soil water volume below which drainage is negligible, and wilting point (soil water volume at ‐1.5 MPa) is the soil water volume below which actual evapotranspiration does not occur (Hillel 1980).</p>
<p>Once available water is calculated, it may exceed total soil storage and become runoff, or it may be less than total soil storage but greater than field capacity and become recharge. Anything less than field capacity will be calculated as actual evapotranspiration at the rate of PET for that month until it reaches wilting point. When soil water is less than total soil storage and greater than field capacity, soil water greater than field capacity equals recharge. If recharge is greater than bedrock permeability (K), then recharge = K and excess becomes runoff, else it will recharge at K until field capacity. Runoff and recharge combine to calculate basin discharge, and actual evapotranspiration is subtracted from PET to calculate climate water deficit.</p>

$$Available\;Soil\;Water_{t} = Available\;Water_{t} + Available\;Soil\;Water_{t-1}$$
$$Available\;Soil\;Water_{t} = Available\;Soil\;Water_{t-1} - Runoff_{t} - Recharge_{t} - AET_{t}$$
* IF Available Soil Water is greater than Total Soil Water then equation 1<br>
* IF Available Soil Water is between Total Soil Water and FC Soil Water then equation 2<br>
* IF Available Soil Water is between FC Soil Water and WLT Soil Water then equation 3<br>
* IF Available Soil Water is less than WLT Soil Water then equation 4<br>
* Current Month Soil Water becomes new Previous Month Soil Water

<strong>Equation 1</strong>
* Runoff = ((Previous Soil Water + Available Water) - Total Soil Water) + (If recharge > geologic K then (Recharge-geologic K)) 
* Recharge = (Total Soil Water - FC Soil Water) 
* AET = PET

<strong>Equation 2</strong>
* Runoff = (If recharge > geologic K then (Recharge-geologic K)) 
* Recharge = Soil Water - FC Soil Water
* AET = PET

<strong>Equation 3</strong>
* Runoff = 0
* Recharge = 0
* AET = PET

<strong>Equation 4</strong>
* Runoff = 0
* Recharge = 0
* AET = 0

http://gis.stackexchange.com/questions/95027/multiple-conditional-statement-with-in-a-con-tool

In [4]:
from arcpy import Raster

In [5]:
monthRange = [1,12] 
yearRange = [2004,2014]

path = "U:/GWP/Groundwater/Projects/BCM/Data/"
field_cap = Raster(path + "Soil.gdb/fieldCap")
wilt_point = Raster(path + "Soil.gdb/WiltPoint")
T_soil_water = Raster(path + "Soil.gdb/Tsoilwater")
geol_k = Raster(path + "Soil.gdb/Geol_K")
#geol_k = Raster(path + "Soil.gdb/BMC_K")
avail_water = "U:/GWP/Groundwater/Projects/BCM/Data/AvailableWater2.gdb/"
pet = path+'MODIS16.gdb/'

results = "U:/GWP/Groundwater/Projects/BCM/Data/Results.gdb/"
    
def UBM_calc(results, field_cap, wilt_point, T_soil_water, geol_k, avail_water, pet, months = '', years = ''):
    
    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True

    if months == '':
        months = [1,12] 
    if years == '':
        years = [2004,2014]

    av_soil_water = field_cap
    for y in range(years[0],years[1]+1): #set years converted here
        for m in range(months[0],months[1]+1): #set months converted here
            my = str(y) + str(m).zfill(2)
            av_wtr = Raster(avail_water+'AVWT'+ my)
            pet_rast = Raster(pet + 'PET'+ my) 

            av_soil_water = av_wtr + av_soil_water

            #Eq 1
            av_recharge1 = "T_soil_water - field_cap"
            recharge1 = "Con(eval(av_recharge1) > geol_k, geol_k, eval(av_recharge1))"
            runoff1 = "(av_soil_water - T_soil_water) + Con(eval(av_recharge1) > geol_k, eval(av_recharge1) - geol_k, 0)"
            #Eq2
            av_recharge2 = "av_soil_water - field_cap"
            recharge2 = "Con(eval(av_recharge2) > geol_k, geol_k, eval(av_recharge2))"
            runoff2 = "Con(eval(av_recharge2) > geol_k, eval(av_recharge2) - geol_k, 0)"
            #Eq3 recharge3 = 0 runoff3 = 0 aet = pet_rast
            #Eq4 recharge3 = 0 runoff3 = 0 aet = 0

            #Order of if/then is Eq 1, Eq 4, Eq 2, Eq 3
            recharge = Con(av_soil_water > T_soil_water, eval(recharge1), 
                      Con(av_soil_water < wilt_point, 0,
                         Con((av_soil_water < T_soil_water) & (av_soil_water > field_cap), eval(recharge2),
                            Con((av_soil_water > wilt_point) & (av_soil_water < field_cap), 0, 0))))
            recharge.save(results + 'rec' + my)

            runoff = Con(av_soil_water > T_soil_water, eval(runoff1), 
                      Con(av_soil_water < wilt_point, 0,
                         Con((av_soil_water < T_soil_water) & (av_soil_water > field_cap), eval(runoff2),
                            Con((av_soil_water > wilt_point) & (av_soil_water < field_cap), 0, 0))))
            runoff.save(results + 'run' + my)

            aet = Con(av_soil_water > T_soil_water, pet_rast, 
                      Con(av_soil_water < wilt_point, 0,
                         Con((av_soil_water < T_soil_water) & (av_soil_water > field_cap), pet_rast,
                            Con((av_soil_water > wilt_point) & (av_soil_water < field_cap), pet_rast,0))))
            aet.save(results + 'aet' + my)

            av_soil_water = av_soil_water - runoff - recharge - aet
            av_soil_water = Con(av_soil_water > 0.0, av_soil_water, 0.0)
            av_soil_water.save(results + 'asw' + my)
            print(my)

200401
200402
200403
200404
200405
200406
200407
200408
200409
200410
200411
200412
200501
200502
200503
200504
200505
200506
200507
200508
200509
200510
200511
200512
200601
200602
200603
200604
200605
200606
200607
200608
200609
200610
200611
200612
200701
200702
200703
200704
200705
200706
200707
200708
200709
200710
200711
200712
200801
200802
200803
200804
200805
200806
200807
200808
200809
200810
200811
200812
200901
200902
200903
200904
200905
200906
200907
200908
200909
200910
200911
200912
201001
201002
201003
201004
201005
201006
201007
201008
201009
201010
201011
201012
201101
201102
201103
201104
201105
201106
201107
201108
201109
201110
201111
201112
201201
201202
201203
201204
201205
201206
201207
201208
201209
201210
201211
201212
201301
201302
201303
201304
201305
201306
201307
201308
201309
201310
201311
201312
201401
201402
201403
201404
201405
201406
201407
201408
201409
201410
201411
201412


In [ ]:
from arcpy import env  
env.overwriteOutput = True  
env.workspace = "C:/Users/PAULINKENBRANDT/AppData/Roaming/ESRI/Desktop10.4/ArcCatalog/AGRC_SGID.sde"

In [35]:
def monthly_to_yearly(path, code, yearRange='', statistics='SUM'):
    if yearRange=='':
        yearRange = [2004,2014]
    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True
    for y in range(yearRange[0],yearRange[1]+1): #set years converted here
        ylist = arcpy.ListRasters(code+str(y)+"*") #pick all files from raw data folder of a data type
        calc = CellStatistics(ylist, statistics_type = statistics, ignore_nodata="DATA")
        outnm = 'y'+code+str(y)
        desc = arcpy.Describe(calc)
        print(outnm)
        calc.save(outnm)


In [38]:
def summarize(path, code, statistics='MEAN'):
    arcpy.env.workspace = path
    arcpy.env.overwriteOutput = True

    rlist = arcpy.ListRasters(code+"*") #pick all files from raw data folder of a data type
    # arcpy sa functions that summarize the daily data to monthly data
    calc = CellStatistics(rlist, statistics_type = statistics, ignore_nodata="DATA")
    outnm = code+"_"+statistics
    calc.save(outnm)
    print(outnm)
    

In [36]:
path = 'H:/GIS/Results.gdb'
code = 'rec'
monthly_to_yearly(path,code)
#summarize(path,code)

(u'RasterDataset', 'yrec2004')
(u'RasterDataset', 'yrec2005')
(u'RasterDataset', 'yrec2006')
(u'RasterDataset', 'yrec2007')
(u'RasterDataset', 'yrec2008')
(u'RasterDataset', 'yrec2009')
(u'RasterDataset', 'yrec2010')
(u'RasterDataset', 'yrec2011')
(u'RasterDataset', 'yrec2012')
(u'RasterDataset', 'yrec2013')
(u'RasterDataset', 'yrec2014')


In [39]:
code = 'yrec'
summarize(path,code)

('yrec_MEAN', <type 'Raster'>)


In [40]:
path = 'H:/GIS/Results.gdb'
code = 'run'
monthly_to_yearly(path,code)
code = 'yrun'
summarize(path,code)

(u'RasterDataset', 'yrun2004')
(u'RasterDataset', 'yrun2005')
(u'RasterDataset', 'yrun2006')
(u'RasterDataset', 'yrun2007')
(u'RasterDataset', 'yrun2008')
(u'RasterDataset', 'yrun2009')
(u'RasterDataset', 'yrun2010')
(u'RasterDataset', 'yrun2011')
(u'RasterDataset', 'yrun2012')
(u'RasterDataset', 'yrun2013')
(u'RasterDataset', 'yrun2014')
('yrun_MEAN', <type 'Raster'>)
